In [1]:
## Import necessary lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
plt.style.use('dark_background')

# from google.colab.patches import cv2_imshow
import cv2
from pdf2image import convert_from_path
import pytesseract as pyt

In [2]:
image_path = './test_data/page59.jpg'

In [3]:
## Read the image and show it on CV
def readCV(path):

    imageRAW = cv2.imread(path, cv2.IMREAD_UNCHANGED)

    return imageRAW

## Show the image in CV2
def showCV(image):

    cv2.imshow('Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


 ## Basic Health of the image
def healthCheck(image):

    print('Here is the Basic Health Check of the Image')
    display('Type of the Image is:',type(image))
    display('Datatype of the Image:',image.dtype)
    display('Shape of the Image:',image.shape)


In [4]:
imgRaw = readCV(image_path)
showCV(imgRaw)
healthCheck(imgRaw)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/Indranath.Chatterjee/ocr_demo/ocr-project-venv/lib/python3.10/site-packages/cv2/qt/plugins"


Here is the Basic Health Check of the Image


'Type of the Image is:'

numpy.ndarray

'Datatype of the Image:'

dtype('uint8')

'Shape of the Image:'

(2299, 1647, 3)

In [5]:
## Extract text from image_opencv
txt_without = pyt.image_to_string(imgRaw)
print(txt_without)

Basic MEpicaL RECORD PROCEDURES
summary is already in the record, it is checked and if it includes all the requested

information, a copy is made. This will save the doctor from having to write a new
report.

Example of format for a summary of medical record information for a medico-legal case:

To: (name of lawyer or law firm requesting information)

Dear.

The following is a summary of the medical record of (patient’s name)
Age____ living at (address)

who was admitted to this hospital on (date of admission)
and who was discharged (or died) on (date of discharge or death)

HISTORY:

PHYSICAL EXAMINATION:

LABORATORY REPORTS:

OPERATION/PROCEDURE

Pathological Report

FINAL DIAGNOSIS:

RESULT ON DISCHARGE:

SIGNED: (Attending Doctor)

53



In [8]:
## Save the file
with open('./extracted_text_from_images/text_extracted_w-o.txt', 'w') as f:
    f.write(txt_without)

## Basic Preprocessing:

In [9]:
## Gray Scale conversion of the Image
def gray(image):

    ## Approach-1: Read images in GrayScale
    # imgGray2 = cv2.imread(samp2, cv2.IMREAD_GRAYSCALE)
    
    ## Approach-2
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return imageGray

## Binary Thresolding
def binThres(image):

    ## Threshold the image to create a binary image  using Otsu’s Binarization

    Thresh, imgBinThres = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    return imgBinThres

 ## Resizing with scale factor
def resizeScaling(image):

    ## As cv2 shape come up with height and width format
    height, width = image.shape[:2]
    scale_factor = float(input('Put the Scale Factor :')) 

    ## Resize the image as per the scale factpr
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    new_size = (new_width, new_height)

    # Resize the image using the new dimensions
    imgScaled = cv2.resize(image, new_size, fx = 1.2, fy = 1.5, interpolation = cv2.INTER_LINEAR)

    ## fx: A scaling factor along the horizontal axis.
    ## fy: A scaling factor along the vertical axis.

    return imgScaled

In [10]:
## Convert image into Grayscale
img_Gray = gray(imgRaw)
showCV(img_Gray)
# healthCheck(img_Gray)

In [11]:
## Binarization
img_Thres_bin = binThres(img_Gray)
showCV(img_Thres_bin)
# healthCheck(img_Thres_bin)

In [12]:
# Denoising of image saving it into dst image 
def noiseRem(image):

    imgDenoise1 = cv2.fastNlMeansDenoising(image, None, 10, 7, 21)

    return imgDenoise1

In [14]:
## Noise Removal
img_denoise = noiseRem(img_Thres_bin)
showCV(img_denoise)
# healthCheck(img_Thres_bin)

In [19]:
## Skew Correction for Text only

def deskew(denoise_img):
    """
    Corrects the skew of an image.
    
    Args:
        image (numpy.ndarray): The image to correct the skew of.
        
    Returns:
        numpy.ndarray: The image with the skew corrected.
    """
    
    edges = cv2.Canny(denoise_img, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi/180, 200)

    # Calculate the angle of the most prominent line
    angles = []
    for line in lines:
        rho, theta = line[0]
        if abs(theta - np.pi/2) > 0.1:
            angle = (theta - np.pi/2)*180/np.pi
            angles.append(angle)

    # Rotate the image to correct the skew
    if len(angles) > 0:
        median_angle = np.median(angles)
        rows, cols = denoise_img.shape[:2]
        M = cv2.getRotationMatrix2D((cols/2, rows/2), -median_angle, 1)
        imageDeskw = cv2.warpAffine(denoise_img, M, (cols, rows), flags=cv2.INTER_CUBIC)

    return imageDeskw

In [20]:
img_deskewed = deskew(img_denoise)
showCV(img_deskewed)
# healthCheck(img_deskewed)

In [21]:
  ## Rotation of the Image

def rotation(image):
    
    ## Approach-1 

    ## getting the first two index of the tuple
    height, width = image.shape[:2]

    # Define the rotation angle in degrees
    angle = float(input('Put the angle of Rotation: '))

    #Calculate the scale factor
    scale_factor = 1.0  # default value i.e no scaling
    # if angle != 0:
    #     scale_factor = abs(math.cos(math.radians(angle))) + abs(math.sin(math.radians(angle)))

    # Calculate the rotation matrix
    center = (width / 2.0, height / 2.0)
    # Calculate the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale_factor)

    # Perform the rotation
    imageRotated = cv2.warpAffine(image, rotation_matrix, (width, height))


    ## Approach-2 when rotate some specific angle

    # imageRotated = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)

    return imageRotated

In [23]:
img_preprocessed = rotation(img_deskewed)
showCV(img_preprocessed)

In [15]:
## Extract text from image_opencv
txt_after_prepro = pyt.image_to_string(img_denoise)
print(txt_after_prepro)

Basic MEpicaL RECORD PROCEDURES
summary is already in the record, it is checked and if it includes all the requested

information, a copy is made. This will save the doctor from having to write a new
report.

Example of format for a summary of medical record information for a medico-legal case:

To: (name of lawyer or law firm requesting information)

Dear.

The following is a summary of the medical record of (patient’s name)
Age_____ living at (address)

who was admitted to this hospital on (date of admission)
and who was discharged (or died) on (date of discharge or death)

HISTORY:

PHYSICAL EXAMINATION:

LABORATORY REPORTS:

OPERATION/PROCEDURE

Pathological Report

FINAL DIAGNOSIS:

RESULT ON DISCHARGE:

SIGNED: (Attending Doctor)

53



In [16]:
## Save the file
with open('./extracted_text_from_images/text_extracted_preprocessed.txt', 'w') as f:
    f.write(txt_after_prepro)